In [28]:
import scanpy as sc
import pandas as pd
import torch

# Metadata: cell-gene pairs
meta_df = pd.read_csv(
    "/maiziezhou_lab2/yunfei/Projects/interpTFM/activations_cosmx_lung_cancer/gene_ids/shard_0/cell_gene_pairs.txt",
    sep="\t", header=None, names=["cell_id", "gene_id"]
)

# Load your main dataframe
f1_df = pd.read_csv(
    "/maiziezhou_lab2/yunfei/Projects/FM_temp/interGFM/output/test/heldout_top_pairings.csv",
    index_col=0
)
f1_df["concept"] = f1_df["concept"].astype(str).str.strip().str.strip('"')
# Load the file with GO terms
go_df = pd.read_csv("/maiziezhou_lab2/yunfei/Projects/interpTFM/gprofiler_annotation/cosmx_lung_human_gp_go_kegg_reactome.csv", usecols=["term_name", "term_id"])

# Merge: match f1_df['concept'] with go_df['term_name']
f1_df = f1_df.merge(go_df, left_on="concept", right_on="term_name", how="left")

# Activations
###### this is wrong, let's try to use the SAE latents to replace thiss
# acts_df = pd.read_parquet("/maiziezhou_lab2/yunfei/Projects/interpTFM/activations_cosmx_lung_cancer/activations/layer_4/shard_0/sae_latents.parquet")
import os
import pandas as pd

base_dir = "/maiziezhou_lab2/yunfei/Projects/interpTFM/activations_cosmx_lung_cancer/activations/layer_4"
shards = [f"shard_{i}" for i in range(60)]

all_cell_ids = set()
missing = []

for sh in shards:
    path = os.path.join(base_dir, sh, "sae_latents.parquet")
    if not os.path.exists(path):
        missing.append(sh)
        continue
    # read only the 'cell_id' column to save memory
    df_ids = pd.read_parquet(path, columns=["cell_id"])
    all_cell_ids.update(df_ids["cell_id"].astype(str).unique())

all_cell_ids = pd.Index(sorted(all_cell_ids))
print(f"Total unique cell_ids across {len(shards)-len(missing)} shards: {len(all_cell_ids):,}")
if missing:
    print("Missing shards (no file found):", missing[:10], "..." if len(missing) > 10 else "")

# Optional: save list
# pd.Series(all_cell_ids, name="cell_id").to_csv("all_cell_ids.csv", index=False)

# Load h5ad to get cell types
adata = sc.read_h5ad("/maiziezhou_lab2/yunfei/Projects/FM_temp/InterPLM/interplm/ge_shards/cosmx_human_lung_sec8.h5ad")
# /maiziezhou_lab2/yunfei/Projects/FM_temp/datasets/cosmx/lung/cosmx_human_lung.h5ad


celltype_map = adata.obs["author_cell_type"].to_dict()  # dict: cell_id → cell_type

# Add cell_type to meta_df
meta_df["cell_type"] = meta_df["cell_id"].map(celltype_map)

Total unique cell_ids across 60 shards: 81,236


In [5]:
acts_df

,cell_id,gene_id,0,1,2,3,4,5,6,7,...,4086,4087,4088,4089,4090,4091,4092,4093,4094,4095
0,96_1-6,<cls>,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,96_1-6,ABL1,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.048794,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,96_1-6,ADGRA3,0.0,0.002219,0.0,0.0,0.0,0.0,0.0,0.067180,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,96_1-6,ADGRE5,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,96_1-6,ADM2,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
516875,3880_20-3,<pad>,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
516876,3880_20-3,<pad>,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
516877,3880_20-3,<pad>,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
516878,3880_20-3,<pad>,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [6]:
f1_df["feature"]

0        208
1       2430
2         71
3         71
4       1725
        ... 
1788    2296
1789    3760
1790    1309
1791     238
1792     238
Name: feature, Length: 1793, dtype: int64

In [ ]:
# # for sanity check of why are we losing cells

# import re
# import pandas as pd
# import numpy as np
# import scanpy as sc

# # ============ 0) helpers ============
# def normalize_feat_name(x):
#     s = str(x).strip()
#     m = re.search(r'(\d+)$', s)
#     if m:
#         try:
#             return int(m.group(1))
#         except ValueError:
#             return s
#     return s

# def peek(name, obj, n=5):
#     try:
#         print(f"[peek] {name} head:")
#         print(obj.head(n))
#     except Exception as e:
#         print(f"[peek] {name} head: <error {e}>")

# # -------- Initial sanity on inputs --------
# print("\n=== INPUT SANITY ===")
# if isinstance(adata, sc.AnnData):
#     print(f"[info] adata: n_obs={adata.n_obs}, n_vars={adata.n_vars}")
# else:
#     print("[warn] 'adata' is not an AnnData")

# print(f"[info] acts_df shape: {acts_df.shape}")
# print(f"[info] acts_df columns (first 12): {list(acts_df.columns)[:12]}")
# print(f"[info] acts_df cell_id nunique: {acts_df['cell_id'].nunique():,}")
# print(f"[info] acts_df example cell_ids: {acts_df['cell_id'].astype(str).unique()[:3]}")
# print(f"[info] acts_df gene_id nunique: {acts_df['gene_id'].nunique():,}")

# # Are there NA cell_ids?
# na_cells = acts_df['cell_id'].isna().sum()
# print(f"[info] acts_df NA cell_id count: {na_cells}")
# if na_cells:
#     print("[warn] NA cell_ids found; those rows will be dropped during groupby/merge.")

# # ============ 1) filter mappings ============
# F1_THRESHOLD = 0.5
# f1_interpretable = f1_df.query("f1 >= @F1_THRESHOLD").copy()
# print(f"\n=== MAPPINGS ===")
# print(f"[info] mappings with F1 ≥ {F1_THRESHOLD}: {len(f1_interpretable):,}")
# peek("f1_interpretable", f1_interpretable)

# if f1_interpretable.empty:
#     raise ValueError("No (feature, concept) pairs passed the F1 threshold.")

# f1_interpretable["feature_norm"] = f1_interpretable["feature"].apply(normalize_feat_name)

# # ============ 2) select feature columns from acts_df ============
# non_feature_cols = {"cell_id", "gene_id"}
# all_cols = list(acts_df.columns)
# feature_cols_raw = [c for c in all_cols if c not in non_feature_cols]
# feature_cols_norm = [normalize_feat_name(c) for c in feature_cols_raw]
# colname_to_norm = dict(zip(feature_cols_raw, feature_cols_norm))

# feat_in_map = set(f1_interpretable["feature_norm"].unique())
# feat_in_acts = set(feature_cols_norm)
# intersection_norm = sorted(feat_in_map.intersection(feat_in_acts))

# print(f"\n=== FEATURE INTERSECTION ===")
# print(f"[info] interpretable features in mapping: {len(feat_in_map):,}")
# print(f"[info] feature columns in acts_df:        {len(feat_in_acts):,}")
# print(f"[info] intersection (used features):      {len(intersection_norm):,}")
# if not intersection_norm:
#     print("[warn] Example mapped feats:", list(feat_in_map)[:10])
#     print("[warn] Example acts_df feats (norm):", list(feat_in_acts)[:10])
#     raise ValueError("Feature/column mismatch after normalization.")

# keep_feature_cols = [orig for orig, norm in colname_to_norm.items() if norm in intersection_norm]

# print(f"[info] keeping {len(keep_feature_cols):,} feature columns (out of {len(feature_cols_raw):,})")
# if len(keep_feature_cols) < 10:
#     print("[peek] keep_feature_cols:", keep_feature_cols)

# # ============ 3) melt acts_df on kept feature columns ============
# print("\n=== MELT ===")
# acts_sel = acts_df[["cell_id", "gene_id"] + keep_feature_cols].copy()

# # Basic sanity of tokens per cell (before melt)
# tok_per_cell = acts_sel.groupby("cell_id", sort=False)["gene_id"].size()
# print(f"[info] tokens per cell: mean={tok_per_cell.mean():.1f}, median={tok_per_cell.median():.1f}, min={tok_per_cell.min()}, max={tok_per_cell.max()}")

# acts_long = (
#     acts_sel
#     .melt(id_vars=["cell_id", "gene_id"], var_name="feature_col", value_name="activation")
# )
# acts_long["feature_norm"] = acts_long["feature_col"].map(colname_to_norm)

# print(f"[info] acts_long shape: {acts_long.shape}")
# print(f"[info] acts_long cells nunique: {acts_long['cell_id'].nunique():,}")
# print(f"[info] acts_long features nunique: {acts_long['feature_norm'].nunique():,}")
# peek("acts_long", acts_long)

# # ============ 4) join with (feature -> concepts) mappings ============
# print("\n=== MERGE FEATURES→CONCEPTS ===")
# map_small = f1_interpretable[["feature_norm", "concept", "f1"]].copy()
# acts_concept_df = acts_long.merge(map_small, on="feature_norm", how="inner")

# print(f"[info] rows after merge: {len(acts_concept_df):,}")
# print(f"[info] cells after merge (nunique): {acts_concept_df['cell_id'].nunique():,}")
# print(f"[info] concepts used: {acts_concept_df['concept'].nunique():,}")
# # which concepts are most frequent?
# print("[info] top concepts by count:")
# print(acts_concept_df["concept"].value_counts().head(10))

# if acts_concept_df.empty:
#     raise ValueError("Empty after merge — check intersections above.")

# # ============ 5) aggregate to cell × concept ============
# print("\n=== AGGREGATE TO CELL × CONCEPT ===")
# concept_matrix = (
#     acts_concept_df
#     .groupby(["cell_id", "concept"], sort=False)["activation"]
#     .mean()
#     .unstack(fill_value=0)
#     .sort_index(axis=0)
#     .sort_index(axis=1)
# )
# print(f"[info] concept_matrix shape (cells × concepts): {concept_matrix.shape}")
# print(f"[info] concept_matrix cell_id nunique: {concept_matrix.index.nunique():,}")
# print(f"[info] concept_matrix concepts: {concept_matrix.shape[1]:,}")

# # optional: how many cells are all-zero across concepts (no interpretable features fired)?
# zero_rows = (concept_matrix.sum(axis=1) == 0).sum()
# print(f"[info] cells with all-zero concept vector: {zero_rows:,} ({zero_rows/len(concept_matrix):.2%})")

# # ============ 6) build new AnnData with concept features ============
# print("\n=== BUILD NEW AnnData ===")
# if isinstance(adata, sc.AnnData):
#     # ensure same dtype for matching (string)
#     acts_cells = concept_matrix.index.astype(str)
#     adata_cells = adata.obs_names.astype(str)

#     common_cells = adata_cells.intersection(acts_cells)
#     print(f"[info] adata cells total: {adata.n_obs:,}")
#     print(f"[info] cells in concept_matrix: {concept_matrix.index.nunique():,}")
#     print(f"[info] cells in intersection:  {len(common_cells):,}")

#     # show a few examples of cells that failed to match (if large drop)
#     if len(common_cells) < concept_matrix.index.nunique():
#         diff = set(acts_cells).difference(set(common_cells))
#         print(f"[warn] {len(diff):,} cells in concept_matrix not found in adata.obs_names (showing up to 5):")
#         print(list(diff)[:5])

#     # align concept_matrix order to adata
#     concept_matrix = concept_matrix.loc[concept_matrix.index.astype(str).isin(common_cells)]
#     concept_matrix = concept_matrix.reindex(adata.obs_names.astype(str).intersection(concept_matrix.index.astype(str)))

#     adata_concept = sc.AnnData(
#         X=concept_matrix.values,
#         obs=adata.obs.loc[concept_matrix.index].copy(),
#         var=pd.DataFrame(index=concept_matrix.columns)
#     )
# else:
#     print("[warn] 'adata' not provided as AnnData; building from concept_matrix only.")
#     adata_concept = sc.AnnData(
#         X=concept_matrix.values,
#         obs=pd.DataFrame(index=concept_matrix.index),
#         var=pd.DataFrame(index=concept_matrix.columns)
#     )

# adata_concept.uns["concept_feature_build"] = {
#     "F1_THRESHOLD": F1_THRESHOLD,
#     "aggfunc": "mean",
#     "n_cells": adata_concept.n_obs,
#     "n_concepts": adata_concept.n_vars,
# }

# print("\n=== SUMMARY ===")
# print(f"[ok] adata_concept: n_obs={adata_concept.n_obs:,}, n_vars={adata_concept.n_vars:,}")

In [30]:
import os, re
import pandas as pd
import numpy as np
import scanpy as sc

# ---- helpers ----
def normalize_feat_name(x):
    s = str(x).strip()
    m = re.search(r'(\d+)$', s)
    if m:
        try: return int(m.group(1))
        except ValueError: return s
    return s

F1_THRESHOLD = 0.5
f1_interpretable = f1_df.query("f1 >= @F1_THRESHOLD").copy()
f1_interpretable["feature_norm"] = f1_interpretable["feature"].apply(normalize_feat_name)
map_small = f1_interpretable[["feature_norm", "concept", "f1"]].copy()

def shard_concept_matrix(latents_df: pd.DataFrame) -> pd.DataFrame:
    """latents_df has columns: cell_id, gene_id, 0..D-1 (latent dims)"""
    non_feature = {"cell_id", "gene_id"}
    feat_cols_raw = [c for c in latents_df.columns if c not in non_feature]
    feat_cols_norm = [normalize_feat_name(c) for c in feat_cols_raw]
    colname_to_norm = dict(zip(feat_cols_raw, feat_cols_norm))

    feat_in_map = set(map_small["feature_norm"])
    inter = sorted(set(feat_cols_norm).intersection(feat_in_map))
    if not inter:
        return None

    keep_cols = [orig for orig, norm in colname_to_norm.items() if norm in inter]
    df = latents_df[["cell_id", "gene_id"] + keep_cols].copy()

    long = df.melt(id_vars=["cell_id", "gene_id"], var_name="feature_col", value_name="activation")
    long["feature_norm"] = long["feature_col"].map(colname_to_norm)

    merged = long.merge(map_small, on="feature_norm", how="inner")
    if merged.empty:
        return None

    # aggregate over tokens and features → cell × concept
    mat = (
        merged.groupby(["cell_id", "concept"])["activation"]
              .mean()                        # or .sum() if you prefer accumulation
              .unstack(fill_value=0)
    )
    return mat

# ---- iterate shards ----
base_dir = "/maiziezhou_lab2/yunfei/Projects/interpTFM/activations_cosmx_lung_cancer/activations/layer_4"
shards = [f"shard_{i}" for i in range(60)]
concept_mats = []
missing = []

for sh in shards:
    path = os.path.join(base_dir, sh, "sae_latents.parquet")
    if not os.path.exists(path):
        missing.append(sh)
        continue
    latents_df = pd.read_parquet(path)  # this includes cell_id, gene_id, and latent dims
    cm = shard_concept_matrix(latents_df)
    if cm is not None and len(cm):
        concept_mats.append(cm)
        print(f"{sh}: cells={cm.shape[0]:,}, concepts={cm.shape[1]:,}")
    else:
        print(f"{sh}: no interpretable features matched; skipping")

# combine all cells
full_concept = pd.concat(concept_mats, axis=0)
full_concept = full_concept[~full_concept.index.duplicated(keep="first")]
print(f"\nFull concept matrix: cells={full_concept.shape[0]:,}, concepts={full_concept.shape[1]:,}")
if missing:
    print("Missing shards:", missing[:10], "..." if len(missing) > 10 else "")

# ---- build AnnData aligned to original adata (optional) ----
common = adata.obs_names.astype(str).intersection(full_concept.index.astype(str))
full_concept = full_concept.loc[common].reindex(adata.obs_names.astype(str).intersection(common))

adata_concept = sc.AnnData(
    X=full_concept.values,
    obs=adata.obs.loc[full_concept.index].copy(),
    var=pd.DataFrame(index=full_concept.columns),
)
adata_concept.uns["concept_feature_build"] = {
    "F1_THRESHOLD": F1_THRESHOLD,
    "n_cells": adata_concept.n_obs,
    "n_concepts": adata_concept.n_vars,
    "n_shards": len(shards),
}

# optional save
# adata_concept.write_h5ad("/maiziezhou_lab2/yunfei/Projects/interpTFM/adata_interpretable_concepts_full.h5ad")

shard_0: cells=1,349, concepts=393
shard_1: cells=1,407, concepts=393
shard_2: cells=1,337, concepts=393
shard_3: cells=1,352, concepts=393
shard_4: cells=1,334, concepts=393
shard_5: cells=1,350, concepts=393
shard_6: cells=1,258, concepts=393
shard_7: cells=1,393, concepts=393
shard_8: cells=1,327, concepts=393
shard_9: cells=1,325, concepts=393
shard_10: cells=1,381, concepts=393
shard_11: cells=1,422, concepts=393
shard_12: cells=1,362, concepts=393
shard_13: cells=1,358, concepts=393
shard_14: cells=1,284, concepts=393
shard_15: cells=1,339, concepts=393
shard_16: cells=1,352, concepts=393
shard_17: cells=1,354, concepts=393
shard_18: cells=1,392, concepts=393
shard_19: cells=1,324, concepts=393
shard_20: cells=1,349, concepts=393
shard_21: cells=1,369, concepts=393
shard_22: cells=1,306, concepts=393
shard_23: cells=1,374, concepts=393
shard_24: cells=1,381, concepts=393
shard_25: cells=1,318, concepts=393
shard_26: cells=1,384, concepts=393
shard_27: cells=1,359, concepts=393
sh

In [32]:
adata_concept

AnnData object with n_obs × n_vars = 81236 × 393
    obs: 'AspectRatio', 'CenterX_global_px', 'CenterY_global_px', 'Width', 'Height', 'Mean.MembraneStain', 'Max.MembraneStain', 'Mean.PanCK', 'Max.PanCK', 'Mean.CD45', 'Max.CD45', 'Mean.CD3', 'Max.CD3', 'Mean.DAPI', 'Max.DAPI', 'niche', 'image_id', 'cell_ID', 'sex_ontology_term_id', 'assay_ontology_term_id', 'organism_ontology_term_id', 'tissue_ontology_term_id', 'suspension_type', 'tissue_type', 'condition_id', 'sample_id', 'donor_id', 'author_cell_type', 'library_key', 'region', 'assay', 'organism', 'sex', 'tissue', 'dataset', 'x', 'y', 'nicheformer_split', '_scvi_batch', '_scvi_labels', 'shards'
    uns: 'concept_feature_build'

In [31]:
# optional save
adata_concept.write_h5ad("/maiziezhou_lab2/yunfei/Projects/interpTFM/evaluation/ccc/data/adata_interpretable_concepts_sec8.h5ad")

In [1]:
import scanpy as sc

adata_concept = sc.read_h5ad("/maiziezhou_lab2/yunfei/Projects/interpTFM/evaluation/ccc/data/adata_interpretable_concepts.h5ad")

In [10]:
adata_concept

AnnData object with n_obs × n_vars = 1349 × 393
    obs: 'AspectRatio', 'CenterX_global_px', 'CenterY_global_px', 'Width', 'Height', 'Mean.MembraneStain', 'Max.MembraneStain', 'Mean.PanCK', 'Max.PanCK', 'Mean.CD45', 'Max.CD45', 'Mean.CD3', 'Max.CD3', 'Mean.DAPI', 'Max.DAPI', 'niche', 'image_id', 'cell_ID', 'sex_ontology_term_id', 'assay_ontology_term_id', 'organism_ontology_term_id', 'tissue_ontology_term_id', 'suspension_type', 'tissue_type', 'condition_id', 'sample_id', 'donor_id', 'author_cell_type', 'library_key', 'region', 'assay', 'organism', 'sex', 'tissue', 'dataset', 'x', 'y', 'nicheformer_split', '_scvi_batch', '_scvi_labels', 'shards'
    uns: 'concept_feature_build'

In [3]:
adata_concept.var

""
concept
"CD4-positive, alpha-beta T cell differentiation involved in immune response"
"CD4-positive, alpha-beta T cell proliferation"
"CD8-positive, alpha-beta T cell activation"
Fc-gamma receptor signaling pathway
G protein-coupled receptor binding
...
tumor necrosis factor receptor activity
type II transforming growth factor beta receptor binding
vascular process in circulatory system


In [5]:
adata_concept.X.max()

2.7552874

In [9]:
import numpy as np
np.count_nonzero(adata_concept.X)

89950

In [11]:
89950 / (1349*393)

0.16966672136744398

In [6]:
adata_concept.X.min()

0.0

# if we want full lung cancer data with more sections

In [12]:
# Load h5ad to get cell types
adata = sc.read_h5ad("/maiziezhou_lab2/yunfei/Projects/FM_temp/datasets/cosmx/lung/cosmx_human_lung.h5ad")
# /maiziezhou_lab2/yunfei/Projects/FM_temp/datasets/cosmx/lung/cosmx_human_lung.h5ad


In [16]:
adata.obs

,AspectRatio,CenterX_global_px,CenterY_global_px,Width,Height,Mean.MembraneStain,Max.MembraneStain,Mean.PanCK,Max.PanCK,Mean.CD45,...,assay,organism,sex,tissue,dataset,x,y,nicheformer_split,_scvi_batch,_scvi_labels
1_1,1.34,4215.888889,158847.666667,47,35,3473,7354,715,5755,361,...,NanoString digital spatial profiling,Homo sapiens,female,lung,nanostring_cosmx_human_lung,4215.888889,158847.666667,train,0,0
2_1,1.45,6092.888889,158834.666667,87,60,3895,13832,18374,53158,260,...,NanoString digital spatial profiling,Homo sapiens,female,lung,nanostring_cosmx_human_lung,6092.888889,158834.666667,train,0,0
3_1,1.62,7214.888889,158843.666667,68,42,2892,6048,3265,37522,378,...,NanoString digital spatial profiling,Homo sapiens,female,lung,nanostring_cosmx_human_lung,7214.888889,158843.666667,train,0,0
4_1,0.47,7418.888889,158813.666667,48,102,6189,16091,485,964,679,...,NanoString digital spatial profiling,Homo sapiens,female,lung,nanostring_cosmx_human_lung,7418.888889,158813.666667,train,0,0
5_1,1.00,7446.888889,158845.666667,38,38,8138,19281,549,874,566,...,NanoString digital spatial profiling,Homo sapiens,female,lung,nanostring_cosmx_human_lung,7446.888889,158845.666667,train,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3999_20-2,1.02,-18135.000000,10850.777778,45,44,7430,9572,370,962,450,...,NanoString digital spatial profiling,Homo sapiens,male,lung,nanostring_cosmx_human_lung,-18135.000000,10850.777778,train,0,0
4000_20-2,1.71,-18088.000000,10846.777778,60,35,8362,11209,161,2024,572,...,NanoString digital spatial profiling,Homo sapiens,male,lung,nanostring_cosmx_human_lung,-18088.000000,10846.777778,train,0,0
4001_20-2,2.75,-19112.000000,10846.777778,99,36,5158,10180,634,2166,41,...,NanoString digital spatial profiling,Homo sapiens,male,lung,nanostring_cosmx_human_lung,-19112.000000,10846.777778,train,0,0
4003_20-2,2.12,-19551.000000,10841.777778,55,26,6339,9804,211,570,488,...,NanoString digital spatial profiling,Homo sapiens,male,lung,nanostring_cosmx_human_lung,-19551.000000,10841.777778,train,0,0


In [15]:
adata.obs.columns

Index(['AspectRatio', 'CenterX_global_px', 'CenterY_global_px', 'Width',
       'Height', 'Mean.MembraneStain', 'Max.MembraneStain', 'Mean.PanCK',
       'Max.PanCK', 'Mean.CD45', 'Max.CD45', 'Mean.CD3', 'Max.CD3',
       'Mean.DAPI', 'Max.DAPI', 'niche', 'image_id', 'cell_ID',
       'sex_ontology_term_id', 'assay_ontology_term_id',
       'organism_ontology_term_id', 'tissue_ontology_term_id',
       'suspension_type', 'tissue_type', 'condition_id', 'sample_id',
       'donor_id', 'author_cell_type', 'library_key', 'region', 'assay',
       'organism', 'sex', 'tissue', 'dataset', 'x', 'y', 'nicheformer_split',
       '_scvi_batch', '_scvi_labels'],
      dtype='object')

In [22]:
adata.obs['image_id'].unique()

array([0, 1, 2, 3, 4, 5, 6, 7])

In [23]:
adata_subset8 = adata[adata.obs['image_id'] == 7]

In [24]:
adata_subset8

View of AnnData object with n_obs × n_vars = 81236 × 960
    obs: 'AspectRatio', 'CenterX_global_px', 'CenterY_global_px', 'Width', 'Height', 'Mean.MembraneStain', 'Max.MembraneStain', 'Mean.PanCK', 'Max.PanCK', 'Mean.CD45', 'Max.CD45', 'Mean.CD3', 'Max.CD3', 'Mean.DAPI', 'Max.DAPI', 'niche', 'image_id', 'cell_ID', 'sex_ontology_term_id', 'assay_ontology_term_id', 'organism_ontology_term_id', 'tissue_ontology_term_id', 'suspension_type', 'tissue_type', 'condition_id', 'sample_id', 'donor_id', 'author_cell_type', 'library_key', 'region', 'assay', 'organism', 'sex', 'tissue', 'dataset', 'x', 'y', 'nicheformer_split', '_scvi_batch', '_scvi_labels'
    var: 'level_0', 'index', 'feature_is_filtered', 'feature_name', 'feature_reference', 'feature_biotype'
    uns: '_scvi_manager_uuid', '_scvi_uuid', 'log1p', 'niche', 'nicheformer_version', 'schema_version', 'title'
    obsm: 'X_niche_0', 'X_niche_1', 'X_niche_2', 'X_niche_3', 'X_niche_4', 'X_pca', 'X_scvi', 'spatial'
    layers: 'log1p', 'ra

In [26]:
adata

AnnData object with n_obs × n_vars = 81236 × 960
    obs: 'AspectRatio', 'CenterX_global_px', 'CenterY_global_px', 'Width', 'Height', 'Mean.MembraneStain', 'Max.MembraneStain', 'Mean.PanCK', 'Max.PanCK', 'Mean.CD45', 'Max.CD45', 'Mean.CD3', 'Max.CD3', 'Mean.DAPI', 'Max.DAPI', 'niche', 'image_id', 'cell_ID', 'sex_ontology_term_id', 'assay_ontology_term_id', 'organism_ontology_term_id', 'tissue_ontology_term_id', 'suspension_type', 'tissue_type', 'condition_id', 'sample_id', 'donor_id', 'author_cell_type', 'library_key', 'region', 'assay', 'organism', 'sex', 'tissue', 'dataset', 'x', 'y', 'nicheformer_split', '_scvi_batch', '_scvi_labels', 'shards'
    var: 'level_0', 'index', 'feature_is_filtered', 'feature_name', 'feature_reference', 'feature_biotype'
    uns: '_scvi_manager_uuid', '_scvi_uuid', 'log1p', 'niche', 'nicheformer_version', 'schema_version', 'title'
    obsm: 'X_niche_0', 'X_niche_1', 'X_niche_2', 'X_niche_3', 'X_niche_4', 'X_pca', 'X_scvi', 'spatial'
    layers: 'log1p', '